In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth

import time
import random
import string
import json

import pandas as pd
import numpy as np

In [2]:
def make_password():
    return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=12))

In [3]:
# options = webdriver.ChromeOptions()
# options.add_argument("--no-sandbox");
# options.add_argument("--disable-dev-shm-usage");
# options.add_argument("--disable-blink-features");

# options.setExperimentalOption("excludeSwitches", Arrays.asList("enable-automation"));

# options.add_argument("--disable-blink-features=AutomationControlled");
# options.add_argument("--disable-infobars");

# driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})");
# driver.delete_all_cookies()

In [4]:
mails = pd.read_csv("../data/mails.csv")

In [5]:
videos = pd.read_csv("../data/uploads.csv")

In [6]:
def vid_to_watch(videos):
    
    to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
    
    videos = videos[videos["video_id"] != to_watch]
    
    return to_watch, videos

In [ ]:
# Add options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-infobars")

options.add_argument("start-maximized")

# options.add_argument("--headless")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Start driver
driver = webdriver.Chrome(options=options)

# Hide the fact we're using a bot
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

# login on google
driver.get("https://accounts.google.com/ServiceLogin")
time.sleep(np.random.uniform(1, 2.5))

email = mails.iloc[0]["mail"]

for letter in email:
    driver.find_element_by_id("identifierId").send_keys(letter)
    time.sleep(np.random.uniform(0.05, 0.2))

driver.find_element_by_id("identifierNext").click()

time.sleep(np.random.uniform(1, 2))

driver.find_element_by_xpath("//input[@name = 'password']").send_keys(mails.iloc[0]["password"])
driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

time.sleep(np.random.uniform(1, 2))

# If security-check is asked, skip it
try:
    driver.find_element_by_xpath("//div[contains(@class, 'U26fgb O0WRkf')]").click()
except:
    pass


# Start watching videos
for i in range(2):
    # Get a random video from the dataset and watch it
    to_watch, videos = vid_to_watch(videos)
    driver.get(f"http://youtube.com/watch?v={to_watch}")
    vid_running = True
    
    # Skip ad(s)
    for i in range(2):
        time.sleep(7)
        try:
            driver.find_element_by_xpath("//div[@class = 'ytp-ad-text ytp-ad-skip-button-text']").click()
        except:
            pass
        
    # Toggle autoplayer
    time.sleep(2)
    autoplay = driver.find_element_by_xpath("//div[@class = 'ytp-autonav-toggle-button']")
        
    if autoplay.get_attribute("aria-checked") == "true":
        autoplay.click()
    
    # Watch video and check if it si done every second
    while vid_running:
        try:
            driver.find_element_by_xpath("//div[contains(@class, 'ended-mode')]")
            vid_running = False
            print("Done watching video...")
        except:
            pass
        
        time.sleep(1)
    
    # Go to the youtube homepage
    driver.get("http://youtube.com")
    time.sleep(1)
    
    # Get videos on youtube home
    recs = driver.find_elements_by_xpath("//div[@id = 'content']//div[@id = 'dismissable']//a")
    
    for element in recs:
        # Alles boven: ytd-rich-section-renderer
        print(element.get_attribute("href"))

Done watching video...
None
https://www.googleadservices.com/pagead/aclk?sa=L&ai=C-P--XNMvYIq-IbmS7_UPp6qR8AvCktu3YdLg3pmXDMXK6KHBIRABIABgkYSAgPwXggEXY2EtcHViLTYyMTk4MTE3NDcwNDkzNzGgAbzCpv4DqQK2YGjygF-0PqgDBMgDCqoExwFP0K134A9JRSC8Z5HcQeflT93T9E1tPbPDnT5L7Fr5UaMHE0mUb_O5Tb6ETsQHR_OcmcV8fff_flbuh7M3-rEn46s2F_mpHxvKooiJ1_o0N4YyDsAWXgxrSmVv3eaHOwcaj7_MY4keftegfX-lHMFR3U_XzEYO0dfUKD-axL14-YiqNUvzyZqVxU9PukkBbbwqOp9IO9_uBFn_TCZ6a4__vNH1aeVjbSt8g1i9YPJ9mQ6OcVg9DqxQXi4YbvQDf7jNkka-qNUHwAS9yJzIoQOSBQIIBZIFCQgTaAF4xKCFA6AGboAHrL3ZAYgHAZAHAqgHhAioB6jSG6gHtgeoB-DPG6gH6dQbqAe21xuoB4zNG6gHsdwbqAfw2RuoB4HGG6gHmM4bqAerxRuoB97OG9IIBwiAgIAQEAKaCTRodHRwczovL3d3dy5lc2FkZS5lZHUvbGFuZGluZ3MvZXZlbnRzLW9ubGluZS1kZWdyZWUvsQm7HKbwBi3fy8gJALoLGwgCEAIYBCAIKAEwA0ABSABYJWAAaABwAYgBANALEuALAbgMAbgT____________AYgUArAUAsAVgYCAQNgVAeIWAggBoBcB&num=1&cid=CAESD-D2GANiaZ3-KEQ4GtvQFw&sig=AOD64_0BjrxV6EnU-fiMmKE88rdzC9ew1g&adurl=https://monitor.ppcprotect.com/v1.0/template%3Faccid%3D13104%26kw%3D%26mt%3D%26nw%3D%26cpn%3D11

In [17]:
need_to_verify = True
attempts = 0

while need_to_verify:
    driver = webdriver.Chrome(ChromeDriverManager().install());

    attempts += 1
    
    fname = names.get_first_name()
    lname = names.get_last_name()
    password = make_password()

    # Visit signup page
    driver.get("http://youtube.com")
    
    

    try:
        driver.find_element_by_xpath('//a[contains(@href,"ServiceLogin")]').click()
    except:
        time.sleep(1.5)
        driver.find_element_by_xpath("//div[@id = 'action-button']").click()

    time.sleep(1)    
    driver.find_element_by_xpath("//div[contains(@class, 'U26fgb c7fp5b')]").click()
    time.sleep(0.5)
    driver.find_element_by_xpath("//div[@class = 'jO7h3c']").click()
    time.sleep(1)

    # Fill in account information
    driver.find_element_by_xpath('//input[@name = "firstName"]').send_keys(fname)
    driver.find_element_by_xpath('//input[@name = "lastName"]').send_keys(lname)
    driver.find_element_by_xpath('//div[@class = "VfPpkd-RLmnJb"]').click()
    driver.find_element_by_xpath('//input[@name = "Username"]').send_keys(fname + lname + str(np.random.randint(1000, 100000)))
    driver.find_element_by_xpath('//input[@name = "Passwd"]').send_keys(password)
    driver.find_element_by_xpath('//input[@name = "ConfirmPasswd"]').send_keys(password)
    driver.find_element_by_xpath('//div[@id = "accountDetailsNext"]').click()
    time.sleep(1)

    try:
        driver.find_element_by_xpath('//span[text() = "Verify your phone number"]')
        print(f"Verification required, retrying (attempt {attempts})")
        driver.quit()
    except Exception as e:
        print(e)
        need_to_verify = False
        

# Fill in phone number (only when asked)
# driver.find_element_by_xpath('//input[@id = "phoneNumberId"]').send_keys("641493584") 
# driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

# gcode = input("G-code (6 digits): ")

# time.sleep(1)
# driver.find_element_by_xpath('//input[@id = "code"]').send_keys(gcode)
# driver.find_element_by_xpath('//div[contains(@class, "VfPpkd-RLmnJb")]').click()
# time.sleep(1)

# Add required personal information
driver.find_element_by_xpath('//input[@id = "day"]').send_keys(np.random.randint(1, 29))
driver.find_element_by_xpath(f"//select[@id='month']/option[@value='{np.random.randint(1, 13)}']").click()
driver.find_element_by_xpath('//input[@id = "year"]').send_keys(np.random.randint(1980, 2001))

driver.find_element_by_xpath(f"//select[@id='gender']/option[@value='{np.random.randint(1, 3)}']").click()
driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()
time.sleep(1)

# Personalisation settings
driver.find_element_by_xpath("//div[contains(@class, 'sSzDje')]").click()
driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()
time.sleep(1)

# Cookies and whatnot
driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()
time.sleep(1)

# Privacy policy
driver.find_element_by_xpath('//div[@id = "termsofserviceNext"]').click()


to_watch, videos = vid_to_watch(videos)


time.sleep(1)
driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

time.sleep(1)
driver.find_element_by_xpath('//input[@name = "password"]').send_keys(password)


    
driver.get(f"http://youtube.com/watch?v={to_watch}")





# # driver.find_element_by_xpath('//input[@id = "phoneNumberId"]').send_keys('641493584')

# # Switch to the new window and open URL B
# driver.switch_to.window(driver.window_handles[1])
# browser.get("https://receive-smss.com/sms/31647264567/")

# time.sleep(60)
# driver.get("https://stackoverflow.com/")

# driver.find_element_by_link_text("Log in").click()

# time.sleep(1)
# driver.find_element_by_xpath("//button[@data-provider = 'google']").click()
# time.sleep(1)
# driver.find_element_by_id("identifierId").send_keys("pascale50lentes@gmail.com")
# driver.find_element_by_id("identifierNext").click()
# driver.find_element_by_name("password").send_keys("rickeneva")
# driver.find_element_by_id("passwordNext").click()

# time.sleep(10)





# driver.delete_all_cookies() 

# driver.get("http://youtube.com")

# try:
#     driver.find_element_by_xpath('//a[contains(@href,"ServiceLogin")]').click()
# except:
#     time.sleep(1)
#     driver.find_element_by_xpath("//div[@id = 'action-button']").click()
    
# driver.find_element_by_xpath("//input[@id = 'identifierId']").send_keys("pascale50lentes@gmail.com")

# driver.implicitly_wait(4)

# driver.find_element_by_xpath("//div[@id = 'identifierNext']").click()

# driver.implicitly_wait(4)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Roan\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[text() = "Verify your phone number"]"}
  (Session info: chrome=88.0.4324.150)



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@id = "day"]"}
  (Session info: chrome=88.0.4324.150)
